# Importing Excel Inventory to bw

- based on import-xlsx from Sabrina


In [2]:
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import pandas as pd
from bw2data import Database

17:42:45+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [3]:
bd.projects.set_current("paw_lca")

In [4]:
### Import LCI from Excel into Brightway
########################################################

print(bd.projects)

# Define variables first
projectName = "paw_lca"
database = "paw_db" # check
ei_match = "ecoinvent-3.11-cutoff"  # the EI version in the project  & file
biosphere_match = "ecoinvent-3.11-biosphere"  # the biosphere version in the project  & file
path = "LCI_ Inventories.xlsx"
# r"C:\Users\Asus\OneDrive - Universidade de Santiago de Compostela\07_Papers\Harmonization study\harmonization inventories_importer_db_test.xlsx"
##########################################################

Brightway2 projects manager with 5 objects:
	Bicycle example
	bw25-tuto
	default
	ecoinvent-3.11-cutoff-bw25_premise
	paw_lca
Use `projects.report()` to get a report on all projects.


In [5]:

# Ensure the Brightway project is set
if projectName in bd.projects:
    bd.projects.set_current(projectName)
    print(f"Project {projectName} is set as current project.")
else:
    print(f"Project {projectName} does not exist.")
    exit()
print("Available databases in the project:", list(bd.databases))
# check databases in the project
if database in bd.databases:
    print(f"Database {database} is already in the project. Will add new data to it.")
    user_input = input("Do you want to continue? (y/n): ").strip().lower()
    if user_input == "n":
        exit()
else:
    user_input = (
        input(f"Do you want to import the database {database}? (y/n): ").strip().lower()
    )
    if user_input == "y":
        print(f"Database {database} will be imported.")
    else:
        print("Process stopped by user.")
        exit()

########################################################
# Create the importer
harmonization_db = bi.ExcelImporter(path)

# apply strategies for matching
harmonization_db.apply_strategies()
# match database to itself
harmonization_db.match_database(fields=["name", "unit", "location"])
# match database to ecoinvent
harmonization_db.match_database(
    ei_match, fields=["name", "location", "reference product"]
)

# match database to biopshere flows
harmonization_db.match_database(biosphere_match, fields=["name", "categories"])
# harmonization_db.match_database("pharma-biosphere", fields=["name", "categories", "location"])

# Optionally, you can inspect unmatched exchanges or activities
print(harmonization_db.statistics())

if harmonization_db.all_linked:
    print("All flows are linked. Database will be written.")
    harmonization_db.write_database()
    print("New database created: ", list(bd.databases))

    # Check how many activities were imported
    db = bd.Database(database)
    activity_count = len(list(db))
    print(f"Number of activities imported: {activity_count}")

    # List all activities
    print("Activities in the database:")
    for i, act in enumerate(db, 1):
        print(f"{i}. {act['name']} ({act['location']})")

else:
    print("Some flows are not linked.")
    print("Unlinked flows:")
    for flow in harmonization_db.unlinked:
        print(flow)
    exit()


# Now your database is available in brightway

Project paw_lca is set as current project.
Available databases in the project:

 ['ecoinvent-3.11-biosphere', 'ecoinvent-3.11-cutoff', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2045 2025-11-12']
Database paw_db will be imported.
Extracted 1 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying 

100%|██████████| 11/11 [00:00<00:00, 611.05it/s]


17:42:53+0100 [info     ] Vacuuming database            
Created database: paw_db
New database created:  ['ecoinvent-3.11-biosphere', 'ecoinvent-3.11-cutoff', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-NPi_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg1000_2045 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2035 2025-11-12', 'ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2045 2025-11-12', 'paw_db']
Number of activities imported: 11
Activities in the database:
1. Electricity 230 (RER)
2. Chemicals 102 (RER)
3. sludge incineration (Europe without Switzerland)
4. transport of sludge to incineration (RER)
5. Electricity 102 (RER)
6. secondary treatment (Europe without Switzerland)
7. GAC 230 (RER)
8. transport of fertilizer to soil (RER)
9. Chemicals 128 (RER)
10. Emissions 230 (RER)
11. Chemicals 230 (RER)


In [6]:
# check metadata
bd.Database(database).metadata

{'format': 'Excel spreadsheet',
 'depends': ['ecoinvent-3.11-biosphere', 'ecoinvent-3.11-cutoff'],
 'backend': 'sqlite',
 'number': 11,
 'modified': '2025-11-12T17:42:53.523964',
 'geocollections': [],
 'processed': '2025-11-12T17:45:07.973671',
 'dirty': False,
 'searchable': True}

In [14]:
bd.Database(database).random()
my_node = bd.get_node(code="trans_fer_to_soi")
print(my_node)

'transport of fertilizer to soil' (ton kilometer, RER, None)


In [15]:
sorted(bd.methods)

[('ecoinvent-3.11',
  'CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'climate change',
  'global warming potential (GWP100)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'ecotoxicity: freshwater',
  'freshwater aquatic ecotoxicity (FAETP inf)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'ecotoxicity: marine',
  'marine aquatic ecotoxicity (MAETP inf)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'ecotoxicity: terrestrial',
  'terrestrial ecotoxicity (TETP inf)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'energy resources: non-renewable',
  'abiotic depletion potential (ADP): fossil fuels'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'eutrophication',
  'eutrophication (fate not incl.)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'human toxicity',
  'human toxicity (HTP inf)'),
 ('ecoinvent-3.11',
  'CML v4.8 2016',
  'material resources: metals/minerals',
  'abiotic depletion potential (ADP): elements (ul

In [22]:
categories_in_order = [method for method in bd.methods if method[0] == 'ecoinvent-3.11' and method[2] == 'acidification']

In [23]:
categories_in_order

[('ecoinvent-3.11',
  'CML v4.8 2016',
  'acidification',
  'acidification (incl. fate, average Europe total, A&B)'),
 ('ecoinvent-3.11', 'EF v3.0', 'acidification', 'accumulated exceedance (AE)'),
 ('ecoinvent-3.11', 'EF v3.1', 'acidification', 'accumulated exceedance (AE)'),
 ('ecoinvent-3.11',
  'EN15804+A2 - Core impact categories and indicators',
  'acidification',
  'accumulated exceedance (AE)'),
 ('ecoinvent-3.11',
  'TRACI v2.1',
  'acidification',
  'acidification potential (AP)')]

In [30]:
'paw_db' in bd.databases

True

In [31]:
bd.databases



paw_db = bd.Database('paw_db')

In [35]:
functional_unit = []

for node in paw_db:
    functional_unit.append(node)



In [36]:
functional_unit

['Emissions 230' (cubic meter, RER, None),
 'Chemicals 128' (cubic meter, RER, None),
 'GAC 230' (cubic meter, RER, None),
 'sludge incineration' (kilogram, Europe without Switzerland, None),
 'transport of sludge to incineration' (ton kilometer, RER, None),
 'Electricity 230' (cubic meter, RER, None),
 'Chemicals 230' (cubic meter, RER, None),
 'transport of fertilizer to soil' (ton kilometer, RER, None),
 'Electricity 102' (cubic meter, RER, None),
 'Chemicals 102' (cubic meter, RER, None),
 'secondary treatment' (cubic meter, Europe without Switzerland, None)]

In [43]:
#bd.get_node(name="sludge incineration").id

bd.get_node(id=247038522800988161)


'sludge incineration' (kilogram, Europe without Switzerland, None)

In [55]:
ef_gwp_key = [
    m for m in bd.methods if "climate change" == m[2] and "EF v3.1" == m[1]
].pop()
print(ef_gwp_key)

# We can look at the method details
ef_gwp_method = bd.Method(ef_gwp_key)
print(ef_gwp_method)

('ecoinvent-3.11', 'EF v3.1', 'climate change', 'global warming potential (GWP100)')
Brightway2 Method: ecoinvent-3.11: EF v3.1: climate change: global warming potential (GWP100)


In [56]:
fu = bd.get_node(id=247038522800988161)

In [57]:
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {fu: 1},
    method=ef_gwp_key,
)
my_lca = bc.LCA(demand=my_functional_unit, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()
my_lca.score

-0.08693584089052496

In [ ]:
fu = bd.get_node(id=247038522800988161)
impact_category = bd.methods.random()

fu, method

('sludge incineration' (kilogram, Europe without Switzerland, None),
 ('ecoinvent-3.11',
  'ReCiPe 2016 v1.03, endpoint (E)',
  'ecosystem quality',
  'water use: terrestrial ecosystems'))

In [ ]:


#calculate lc inventory
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs({fu: 1})
my_lca = bc.LCA(demand=my_functional_unit, data_objs=data_objs)
my_lca.lci()

In [ ]:
#calculate chracterized inventory
my_functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {fu: 1},
    method=impact_category
)
my_lca = bc.LCA(demand=my_functional_unit, data_objs=data_objs)
my_lca.lci()
my_lca.lcia()

NameError: name 'impact_category' is not defined

In [ ]:
functional_unit, data_objs, _ = bd.prepare_lca_inputs(
    {functional_unit: 1},  # Here you don't need the `.id`, but you can also use it if you want
    method=('IPCC',), 
)




lca = bc.LCA(
    demand=functional_unit,
    data_objs=data_objs + [dp_correlated],
    use_distributions=True,
    use_arrays=True,
)

TypeError: unhashable type: 'list'